In [ ]:
# Units are mm, N, and MPa (N/mm²)
from dataclasses import dataclass
from typing import List
from Pynite import FEModel3D
from Pynite.Rendering import Renderer


ROOM_WIDTH = 3000  # Along X-axis
ROOM_LENGTH = 1870 # Along Z-axis
ROOM_HEIGHT = 5465 # Along Y-axis
PLANK_THICKNESS = 25

floor2floor = ROOM_HEIGHT/2 + PLANK_THICKNESS/2
wall_beam_contact_depth = 40
beam_length = ROOM_LENGTH + wall_beam_contact_depth

opening_width = 630 # Along the Z-axis
opening_z_start = opening_width + wall_beam_contact_depth/2

frame = FEModel3D()
E = 11000
nu = 0.3
rho = 4.51e-6
frame.add_material('wood', E=E, G=(E / (2 * (1 + nu))), nu=nu, rho=rho)

E = 7000
nu = 0.2
rho = 5.75e-6
frame.add_material('brick', E=E, G=(E / (2 * (1 + nu))), nu=nu, rho=rho)


@dataclass
class BeamSpec:
    template_name: str
    base: int
    height: int
    material: str
    beam_type: str  # 'joist', 'trimmer', 'tail', 'header'
    name: str = ''
    
    @property
    def section_name(self) -> str:
        return f"sec_{self.base}x{self.height}"
    
    def create_section(self, frame: FEModel3D):
        if self.section_name not in frame.sections:
            A = self.base * self.height
            b, h = min(self.base, self.height), max(self.base, self.height)
            J = (b**3 * h) * (1/3 - 0.21 * (b/h) * (1 - (b**4)/(12*h**4)))
            Iy = (self.height * self.base**3) / 12
            Iz = (self.base * self.height**3) / 12
            frame.add_section(self.section_name, A, Iy, Iz, J)

    def copy(self, **kwargs):
        new = BeamSpec(**self.__dict__)
        for k, v in kwargs.items():
            setattr(new, k, v)
        return new

@dataclass
class BeamPlacement:
    spec: BeamSpec
    x_center: float
    z_start: float = 0
    z_end: float = None
    
    def add_to_frame(self, frame: FEModel3D, floor2floor: float, default_z_end: float):
        z_end = self.z_end if self.z_end is not None else default_z_end
        name = self.spec.name
        
        if not name.startswith('tail'):
            frame.add_node(f'floor {name}N', self.x_center, 0, z_end)
        frame.add_node(f'floor {name}S', self.x_center, 0, self.z_start)
        frame.add_node(f'{name}S', self.x_center, floor2floor, self.z_start)
        frame.add_node(f'{name}N', self.x_center, floor2floor, z_end)
        
        self.spec.create_section(frame)
        frame.add_member(name, f'{name}N', f'{name}S', self.spec.material, self.spec.section_name)

class LayoutManager:
    def __init__(self, room_width: float):
        self.room_width = room_width
        self.beams: List[BeamPlacement] = []
        self._is_sorted = False

    def add_beam(self, spec: BeamSpec, x_center: float, z_start: float = 0, z_end: float = None) -> BeamPlacement:
        placement = BeamPlacement(spec, x_center, z_start, z_end)
        self.beams.append(placement)
        self._is_sorted = False
        return placement

    def add_beam_at_offset(self, spec: BeamSpec, x_offset: float, **kwargs) -> BeamPlacement:
        """
        Adds a beam based on the clear offset from the east wall (x=0) to its nearest face.
        
        Example: An offset of 820mm for an 80mm wide beam places its east face at
        x=820 and its centerline at x=860.
        """
        x_center = x_offset + spec.base / 2
        return self.add_beam(spec, x_center, **kwargs)

    def add_beam_between(self, spec: BeamSpec, east_beam: BeamPlacement, west_beam: BeamPlacement, **kwargs) -> BeamPlacement:
        east_beam_west_edge = east_beam.x_center + east_beam.spec.base / 2
        west_beam_east_edge = west_beam.x_center - west_beam.spec.base / 2
        
        clear_span = west_beam_east_edge - east_beam_west_edge
        if spec.base > clear_span:
            raise ValueError(f"Cannot place beam '{spec.name}' ({spec.base}mm wide) in a clear span of only {clear_span:.1f}mm.")
            
        x_center = (east_beam_west_edge + west_beam_east_edge) / 2
        return self.add_beam(spec, x_center, **kwargs)

    def sort_beams(self):
        self.beams.sort(key=lambda p: p.x_center)
        self._is_sorted = True
    
    def apply_dead_loads(self, frame: FEModel3D):
        for p in self.beams:
            section = frame.sections[p.spec.section_name]
            material = frame.materials[p.spec.material]
            dead_load = -section.A * material.rho
            frame.add_member_dist_load(p.spec.name, 'FY', dead_load, dead_load)
    
    def apply_live_loads(self, frame: FEModel3D, live_load_mpa: float, opening_z: float):
        if not self._is_sorted:
            raise RuntimeError("LayoutManager must be sorted before applying loads.")
        
        for i, beam_placement in enumerate(self.beams):
            pos_left = self.beams[i-1].x_center if i > 0 else 0
            pos_right = self.beams[i+1].x_center if i < len(self.beams)-1 else self.room_width
            
            trib_width_left = (beam_placement.x_center - pos_left) / 2
            trib_width_right = (pos_right - beam_placement.x_center) / 2
            
            load_left = live_load_mpa * trib_width_left
            load_right = live_load_mpa * trib_width_right

            if beam_placement.spec.beam_type in ['joist', 'tail']:
                total_load = load_left + load_right
                frame.add_member_dist_load(beam_placement.spec.name, 'FY', total_load, total_load)
            elif beam_placement.spec.beam_type == 'trimmer':
                # For trimmers, one side has a load break at the opening.
                is_left_opening = i > 0 and self.beams[i-1].spec.beam_type == 'tail'
                is_right_opening = i < len(self.beams)-1 and self.beams[i+1].spec.beam_type == 'tail'
                
                if is_left_opening:
                    frame.add_member_dist_load(beam_placement.spec.name, 'FY', load_left, load_left, 0, opening_z)
                    frame.add_member_dist_load(beam_placement.spec.name, 'FY', load_right, load_right)
                elif is_right_opening:
                    frame.add_member_dist_load(beam_placement.spec.name, 'FY', load_left, load_left)
                    frame.add_member_dist_load(beam_placement.spec.name, 'FY', load_right, load_right, 0, opening_z)
                else:
                    frame.add_member_dist_load(beam_placement.spec.name, 'FY', load_left + load_right, load_left + load_right)


joist_spec = BeamSpec('joist', base=60, height=120, material='wood', beam_type='joist')
trimmer_spec = BeamSpec('trimmer', base=80, height=160, material='wood', beam_type='trimmer')
header_spec = BeamSpec('header', base=120, height=120, material='wood', beam_type='header')

layout = LayoutManager(room_width=ROOM_WIDTH)
beam_A = layout.add_beam_at_offset(joist_spec.copy(name='A'), x_offset=0)
trimmer_E = layout.add_beam_at_offset(trimmer_spec.copy(name='trimmer E'), x_offset=820)
beam_B = layout.add_beam_between(joist_spec.copy(name='B'), beam_A, trimmer_E)

tail_length = beam_length - opening_width - wall_beam_contact_depth/2 - joist_spec.base
tail_spec = joist_spec.copy(beam_type='tail')
tail_C = layout.add_beam_at_offset(tail_spec.copy(name='tail E'), x_offset=1295, z_end=tail_length)
tail_D = layout.add_beam_at_offset(tail_spec.copy(name='tail W'), x_offset=1645, z_end=tail_length)

trimmer_W = layout.add_beam_at_offset(trimmer_spec.copy(name='trimmer W'), x_offset=2140)
beam_E = layout.add_beam_at_offset(joist_spec.copy(name='C'), x_offset=ROOM_WIDTH - joist_spec.base)


# Build the Frame Geometry
layout.sort_beams()
for beam_placement in layout.beams:
    beam_placement.add_to_frame(frame, floor2floor, beam_length)

header_spec.create_section(frame)
frame.add_node('header E', trimmer_E.x_center, floor2floor, tail_length)
frame.add_node('header W', trimmer_W.x_center, floor2floor, tail_length)
frame.add_member('header', 'header W', 'header E', header_spec.material, header_spec.section_name)

for node_name, node in frame.nodes.items():
    if node_name.startswith('floor'):
        frame.def_support(node_name, True, True, True, True, True, True)


def auto_add_walls(frame, layout, wall_thickness, material):
    layout.sort_beams()
    
    eastmost_beam = layout.beams[0]  # Beam closest to x=0
    westmost_beam = layout.beams[-1]  # Beam furthest from x=0
    
    def node(floor, beam, end):
        return f"{'floor ' if floor else ''}{beam.spec.name}{end}"
    
    frame.add_quad(
        'west wall',
        node(True, westmost_beam, 'S'),
        node(True, westmost_beam, 'N'),
        node(False, westmost_beam, 'N'),
        node(False, westmost_beam, 'S'),
        wall_thickness,
        material
    )
    frame.add_quad(
        'east wall',
        node(True, eastmost_beam, 'N'),
        node(True, eastmost_beam, 'S'),
        node(False, eastmost_beam, 'S'),
        node(False, eastmost_beam, 'N'),
        wall_thickness,
        material
    )
    
    prev_beam = None
    for i, beam in enumerate(layout.beams):
        if prev_beam is None:
            prev_beam = beam
            continue
        frame.add_quad(
            f'south wall {prev_beam.spec.name}-{beam.spec.name}',
            node(True, prev_beam, 'S'),
            node(True, beam, 'S'),
            node(False, beam, 'S'),
            node(False, prev_beam, 'S'),
            wall_thickness,
            material
        )
        prev_beam = beam
    
    north_reaching_beams = [b for b in layout.beams if b.spec.beam_type != 'tail']
    prev_beam = None
    for beam in north_reaching_beams:
        if prev_beam is None:
            prev_beam = beam
            continue
        frame.add_quad(
            f'north wall {prev_beam.spec.name}-{beam.spec.name}',
            node(True, prev_beam, 'N'),
            node(True, beam, 'N'),
            node(False, beam, 'N'),
            node(False, prev_beam, 'N'),
            wall_thickness,
            material
        )
        prev_beam = beam


def apply_header_dead_load(frame: FEModel3D, header_name: str, header_spec: BeamSpec):
    section = frame.sections[header_spec.section_name]
    material = frame.materials[header_spec.material]
    dead_load = -section.A * material.rho
    frame.add_member_dist_load(header_name, 'FY', dead_load, dead_load)


auto_add_walls(frame, layout, wall_thickness=80, material='brick')

# Apply loads
layout.apply_dead_loads(frame)
apply_header_dead_load(frame, 'header', header_spec)
layout.apply_live_loads(frame, live_load_mpa=-0.003, opening_z=opening_z_start)

# Analyze
frame.analyze(check_statics=True)

# Print results
for beam in frame.members:
    print(f"\n--- {beam} Stats ---")
    print(f"Max Moment (Mz): {frame.members[beam].max_moment('Mz', 'Combo 1'):.3f} N-mm")
    print(f"Min Moment (Mz): {frame.members[beam].min_moment('Mz', 'Combo 1'):.3f} N-mm")
    print(f"Max Shear (Fy): {frame.members[beam].max_shear('Fy', 'Combo 1'):.3f} N")
    print(f"Min Shear (Fy): {frame.members[beam].min_shear('Fy', 'Combo 1'):.3f} N")
    print(f"Max Deflection (dy): {frame.members[beam].max_deflection('dy', 'Combo 1'):.3f} mm")
    print(f"Min Deflection (dy): {frame.members[beam].min_deflection('dy', 'Combo 1'):.3f} mm")


def set_wall_opacity(plotter, opacity=0.5):  
    """Set opacity for wall quads to see through them"""
    for actor in plotter.renderer.actors.values():
        if (hasattr(actor, 'mapper') and
            hasattr(actor.mapper, 'dataset') and
            actor.mapper.dataset.n_faces_strict > 0):
            actor.prop.opacity = opacity

# Render
rndr = Renderer(frame)
rndr.annotation_size = 5
rndr.render_loads = False  # Set to False to avoid rendering stall
rndr.deformed_shape = True
rndr.deformed_scale = 1000
opacity = 0.25
rndr.post_update_callbacks.append(lambda plotter: set_wall_opacity(plotter, opacity=opacity))
rndr.render_model()

In [ ]:
span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
k_mod = 0.8
gamma_M = 1.3
service_deflection_ratio = 300
dynamic_factors = [1.0, 1.2, 1.5]

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08), (0.02, 0.04), (0.01, 0.02)]

wood_grades = {
    "C14 (Softwood)": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    "C18 (Softwood)": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    "C22 (Softwood)": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24 (Softwood)": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    "D30 (Hardwood)": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    "D40 (Hardwood)": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
    "D50 (Hardwood)": {"f_mk": 50, "f_vk": 5.0, "E_mean": 16000e6},
}

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    defl_ok = True
    for dyn in dynamic_factors:
        if not defl_ok: break
        w_total = (w_surface * dyn + beam_weight / span) * 1000
        M_max = w_total * span**2 / 8
        V_max = w_total * span / 2
        I = b * h**3 / 12
        c = h / 2
        sigma_max = M_max * c / I
        Q = b * h**2 / 8
        tau_max = V_max * Q / (I * b)
        A = b * h
        tau_avg = V_max / A
        print(f"\nBeam {b*100:.1f}x{h*100:.1f} cm, Dynamic factor={dyn}")
        print(f"Distributed load: {w_total/1000:.3f} kN/m")
        print(f"Max moment: {M_max/1e3:.3f} kN·m")
        print(f"Max shear: {V_max/1e3:.3f} kN")
        print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
        print(f"Shear stress: {tau_max/1e6:.3f} MPa")
        for grade, props in wood_grades.items():
            f_md = k_mod * props["f_mk"] / gamma_M
            f_vd = k_mod * props["f_vk"] / gamma_M
            E = props["E_mean"]
            delta_max = 5 * w_total * span**4 / (384 * E * I)
            service_limit = span / service_deflection_ratio
            bend_ok = sigma_max/1e6 <= f_md
            shear_ok = tau_max/1e6 <= f_vd
            defl_ok = delta_max <= service_limit
            status = []
            status.append("Bend OK" if bend_ok else "BEND FAIL")
            status.append("Shear OK" if shear_ok else "SHEAR FAIL")
            status.append("Defl OK" if defl_ok else "DEFLECTION FAIL")
            print(f"{grade}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, "
                  f"Defl={delta_max:.3f} m, Status: {', '.join(status)}")


In [ ]:
beam_base = 80
beam_height = 160
beam_length = 1910

beam_base**3 * beam_height * ((1/3) - (0.21 * beam_base/beam_height * (1 - (beam_base ** 4 / (12 * beam_height ** 4)))))

In [ ]:
0.229 * beam_length * beam_base ** 3

In [ ]:
import pandas as pd

# 1. Room & Floor Geometry
room_width_m = 1.87
room_length_m = 3.0

# 2. Loading Conditions
live_loads = range(3,31)
dead_load_planks_q_kPa = 0.25 # Dead load of the floor planks/decking in kN/m^2

# 3. Beam Configuration & Support
number_of_beams = 4
beam_bearing_length_m = 0.04

# 4. Material Properties & Safety Factors
k_mod = 0.8   # Modification factor for load duration and moisture (e.g., 0.8 for service class 2)
gamma_M = 1.3 # Partial safety factor for wood material properties
brick_safety_factor = 5.0 # Higher safety factor for brittle, hollow brick

# 5. Serviceability & Dynamic Limits
service_deflection_ratio = 360 # Stricter deflection limit (L/360 is common)
dynamic_factor = 1.0 # Assuming static load for simplicity. Change if floor will see dynamic use.

beam_sizes_m = [
    # (0.033, 0.07), 
    # (0.045, 0.09), 
    (0.06, 0.12), 
    (0.08, 0.16), 
    # (0.10, 0.20), 
    # (0.12, 0.24), 
]

wood_grades = {
    # "C14": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    # "C18": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    # "C22": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    # "D30": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    # "D40": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
}

wood_species = {
    # "Pinus sylvestris": {"density_kN": 4.707, "durability_DC": 3.5},
    # "Picea abies": {"density_kN": 4.413, "durability_DC": 4},
    "Larix decidua": {"density_kN": 5.884, "durability_DC": 3},
    # "Pinus radiata": {"density_kN": 4.217, "durability_DC": 4.5},
}

brick_types = {
    "Spanish Hollow Brick (Assumed)": {"compressive_strength_MPa": 10.0, "is_hollow": True},
    "Solid Clay Brick (for comparison)": {"compressive_strength_MPa": 20.0, "is_hollow": False},
}
selected_brick = brick_types["Spanish Hollow Brick (Assumed)"]

results = []
span = room_width_m
beam_spacing_m = room_length_m / number_of_beams

for b, h in beam_sizes_m:
    for live_load_q_kPa in live_loads:
        for grade_name, wood_props in wood_grades.items():
            for species_name, species_props in wood_species.items():
                # Self-weight of the beam
                beam_volume = b * h * span
                beam_weight_kN = beam_volume * species_props['density_kN']
                beam_self_weight_dist_kN_m = beam_weight_kN / span
                # Surface loads (live + dead) tributary to one beam
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * beam_spacing_m
                # Total design load on the beam
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000

                # Bending and Shear Forces
                M_max_Nm = w_total_N_m * span**2 / 8
                V_max_N = w_total_N_m * span / 2 # This is also the reaction force at the support
                # Beam geometric properties
                I_m4 = b * h**3 / 12  # Moment of inertia
                A_m2 = b * h           # Cross-sectional area
                # Stresses in the beam
                sigma_max_Pa = (M_max_Nm * (h / 2)) / I_m4
                tau_max_Pa = (3 * V_max_N) / (2 * A_m2) # Max shear for rectangle
                # Allowable design stresses for the wood
                f_md_Pa = (k_mod * wood_props["f_mk"] / gamma_M) * 1e6
                f_vd_Pa = (k_mod * wood_props["f_vk"] / gamma_M) * 1e6
                # Deflection
                E_Pa = wood_props["E_mean"]
                delta_max_m = (5 * w_total_N_m * span**4) / (384 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio

                # Calculate stress on bricks
                bearing_area_m2 = b * beam_bearing_length_m
                bearing_stress_on_brick_Pa = V_max_N / bearing_area_m2
                allowable_brick_stress_Pa = (selected_brick["compressive_strength_MPa"] * 1e6) / brick_safety_factor

                # Failure mode booleans
                bend_ok = sigma_max_Pa <= f_md_Pa
                shear_ok = tau_max_Pa <= f_vd_Pa
                defl_ok = delta_max_m <= service_limit_m
                brick_ok = bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa

                results.append({
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Wood Grade": grade_name,
                    "Wood Species": species_name,
                    "Total Beam Weight": beam_weight_kN * 6,
                    "Beam Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6,
                    "Beam Shear Stress (MPa)": tau_max_Pa / 1e6,
                    "Allowable Shear (MPa)": f_vd_Pa / 1e6,
                    "Live load": live_load_q_kPa,
                    "Deflection (mm)": delta_max_m * 1000,
                    "Allowable Defl (mm)": service_limit_m * 1000,
                    "Brick Bearing Stress (MPa)": bearing_stress_on_brick_Pa / 1e6,
                    "Allowable Brick Stress (MPa)": allowable_brick_stress_Pa / 1e6,
                    "Bend OK": bend_ok,
                    "Shear OK": shear_ok,
                    "Defl OK": defl_ok,
                    "Brick OK": brick_ok
                })

df = pd.DataFrame(results)
df

In [ ]:
from Pynite import FEModel3D
from Pynite.Rendering import Renderer
frame = FEModel3D()

# Constants
room_width = 1870
room_length = 3000
floor_height = 2750
beam_base = 60
beam_height = 120
wall_beam_contact_area = 40
beam_length = room_width + wall_beam_contact_area # NOT full length, calculated brick' center. Full length == room_width + wall_beam_contact_area * 2

# Materials
E = 11000 # MPa (N/mm²)
nu = 0.3
G = E / (2 * (1 + nu))  # MPa (N/mm²)
rho = 6.0e-7  # kg/mm3
frame.add_material('wood', E=E, G=G, nu=nu, rho=rho)

E = 7000 # MPa (N/mm²)
nu = 0.2
G = E / (2 * (1 + nu))  # MPa (N/mm²)
rho = 5.45e-7 # kg/mm3
frame.add_material('brick', E=E, G=G, nu=nu, rho=rho)

# Beam cross-section
A = beam_base * beam_height
J = beam_base ** 3 * beam_height * ((1/3) - (0.21 * beam_base/beam_height * (1 - (beam_base ** 4 / (12 * beam_height ** 4)))))
Iy = (beam_base ** 3 * beam_height) / 12
Iz = (beam_base * beam_height ** 3) / 12
frame.add_section('beam', A, Iy, Iz, J)

# Floor nodes and supports
frame.add_node('floor SE', beam_base/2, 0, 0)
frame.add_node('floor NE', beam_base/2, 0, room_width)
# frame.add_node('floor SW', room_length - beam_base/2, 0, 0)
# frame.add_node('floor NW', room_length - beam_base/2, 0, room_width)

frame.add_member('A', 'floor SE', 'floor NE', 'wood', 'beam')

In [ ]:
frame.members['A'].section.name

In [ ]:
# Constants
room_width = 1870
room_length = 3017
room_height = 5465
plank_thickness = 25
floor2floor = room_height/2 + plank_thickness/2
floor2ceiling = room_height/2 - plank_thickness/2
beam_base = 60
beam_height = 160
floor2beam = floor2ceiling - beam_height
wall_beam_contact_area = 40
beam_length = room_width + wall_beam_contact_area

In [ ]:
floor2ceiling, floor2floor, floor2beam

In [ ]:
E = 11000 # MPa (N/mm²)
nu = 0.3
G = E / (2 * (1 + nu))  # MPa (N/mm²)
G